<a href="https://colab.research.google.com/github/yanmingl/NaturalLanguageProcessing/blob/master/seq2seq_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -V

Python 3.6.13 :: Anaconda, Inc.


In [3]:
import tensorflow as tf
print(tf.__version__)

ModuleNotFoundError: No module named 'tensorflow.python'

In [1]:

import tensorflow as tf
import numpy as np
import os, re
from tensorflow.python.layers.core import Dense

Looking in indexes: http://pypi.douban.com/simple
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4848 sha256=a44e298d85229150a3503db243069ada28b894ab4d13329c40377b43a7677548
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\ca\f2\60\0b9724e1c596aac6849f5240074453c349c267652d0a9a6428
Successfully built termcolor
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 1.4.0
    Uninstalling tensorflow-1.4.0:
      Successfully uninstalled tensorflow-1.4.0


d:\miniconda3\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\miniconda3\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\miniconda3\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\miniconda3\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [2]:
MAX_VOC_PER_LINE = 40

def load_sentences(input_path, output_path):
    with open(input_path, 'r', encoding="ISO-8859-1") as f1, \
         open(output_path, 'r', encoding="ISO-8859-1") as f2:
        input_data, output_data = f1.read().encode('ascii', 'ignore').decode('UTF-8'), \
                                  f2.read().encode('ascii', 'ignore').decode('UTF-8')
        # input_data = re.sub('[^a-z\n]+', ' ', input_data)
        # output_data = re.sub('[^a-z\n]+', ' ', output_data)
        input_seq, output_seq = [], []
        for i, line in enumerate(input_data.split('\n')):
            input_voc_list_per_line = line.split(' ')
            input_length_voc_per_line = len(input_voc_list_per_line)
            if input_length_voc_per_line > MAX_VOC_PER_LINE:
              continue
            output_data_per_line = output_data.split('\n')
            current_output_line = output_data_per_line[i]
            current_output_length_line = len(current_output_line.split(' '))
            if current_output_length_line > MAX_VOC_PER_LINE:
              continue
            input_seq.append(line)
            output_seq.append(current_output_line)

    return input_seq, output_seq

In [3]:
input_sentences, output_sentences = load_sentences('data/words_input.txt', 'data/words_output.txt')  
print(input_sentences[:5], '\n', len(input_sentences))
print(output_sentences[:5], '\n', len(output_sentences))

['can we make this quick ? roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad . again .', 'well , i thought we would start with pronunciation , if that is okay with you .', 'not the hacking and gagging and spitting part . please .', 'you are asking me out . that is so cute . that is your name again ?', 'no , no , it is my fault we did not have a proper introduction'] 
 45609
['well , i thought we would start with pronunciation , if that is okay with you .', 'not the hacking and gagging and spitting part . please .', 'okay . . . then how about we try out some french cuisine . saturday ? night ?', 'forget it .', 'cameron .'] 
 45609


In [4]:
def extract_vocab(data):
    special_vocs = ['<PAD>', '<UNK>', '<GO>',  '<EOS>']
    set_vocs = set([voc for line in data for voc in line.split(' ')])
    all_vocs = special_vocs + list(set_vocs)
    int_to_voc = {word_i: word for word_i, word in enumerate(all_vocs)}
    voc_to_int = {word: word_i for word_i, word in int_to_voc.items()}
    return int_to_voc, voc_to_int


input_int_to_voc, input_voc_to_int = extract_vocab(input_sentences)
output_int_to_voc, output_voc_to_int = extract_vocab(output_sentences)
[['yes', ''], []]

In [6]:
# print(input_int_to_voc)
# print(len(input_int_to_voc))
# print(output_int_to_voc)
# print(len(output_int_to_voc))
# print(input_voc_to_int)

# Here is the puzzle. The number of tokenized word is different with the one of 
# transformer. 

In [7]:
NUM_EPOCS = 3
RNN_STATE_DIM = 512
RNN_NUM_LAYERS = 2
ENCODER_EMBEDDING_DIM = DECODER_EMBEDDING_DIM = 64
 
BATCH_SIZE = int(64)
LEARNING_RATE = 0.0003
 
INPUT_NUM_VOCAB = len(input_voc_to_int)
OUTPUT_NUM_VOCAB = len(output_voc_to_int)
print(OUTPUT_NUM_VOCAB)

16917


In [8]:
# Encoder placeholders
encoder_input_seq = tf.placeholder(
    tf.int32, 
    [None, None],  # batch_size * seq_length
    name='encoder_input_seq'
)

encoder_seq_len = tf.placeholder(
    tf.int32, 
    (None,),   # shape is dynamic bca th length of a sequence can change
    name='encoder_seq_len'
)
 
# Decoder placeholders
decoder_output_seq = tf.placeholder( 
    tf.int32, 
    [None, None],
    name='decoder_output_seq'
)

decoder_seq_len = tf.placeholder(
    tf.int32,
    (None,), 
    name='decoder_seq_len'
)

max_decoder_seq_len = tf.reduce_max(  
    decoder_seq_len,   # maximum length of a decoder sequence for a batch
    name='max_decoder_seq_len'  
)

In [9]:
def make_cell(state_dim):
    lstm_initializer = tf.random_uniform_initializer(-0.1, 0.1)
    return tf.contrib.rnn.LSTMCell(state_dim, initializer=lstm_initializer)
 
def make_multi_cell(state_dim, num_layers):
    cells = [make_cell(state_dim) for _ in range(num_layers)]
    return tf.contrib.rnn.MultiRNNCell(cells)

In [10]:
# Encoder embedding
 
encoder_input_embedded = tf.contrib.layers.embed_sequence(
    encoder_input_seq,     
    INPUT_NUM_VOCAB,        
    ENCODER_EMBEDDING_DIM  
)
 
 
# Encodering output
 
encoder_multi_cell = make_multi_cell(RNN_STATE_DIM, RNN_NUM_LAYERS)
 
encoder_output, encoder_state = tf.nn.dynamic_rnn(
    encoder_multi_cell, 
    encoder_input_embedded, 
    sequence_length=encoder_seq_len, 
    dtype=tf.float32
)
 
del(encoder_output)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where

In [11]:
decoder_raw_seq = decoder_output_seq[:, :-1]  
go_prefixes = tf.fill([BATCH_SIZE, 1], output_voc_to_int['<GO>'])  
decoder_input_seq = tf.concat([go_prefixes, decoder_raw_seq], 1)  

In [12]:
decoder_embedding = tf.Variable(tf.random_uniform([OUTPUT_NUM_VOCAB, 
                                                   DECODER_EMBEDDING_DIM]))
decoder_input_embedded = tf.nn.embedding_lookup(decoder_embedding, 
                                                decoder_input_seq)

decoder_multi_cell = make_multi_cell(RNN_STATE_DIM, RNN_NUM_LAYERS)
 
output_layer_kernel_initializer = tf.truncated_normal_initializer(mean=0.0, stddev=0.1)
output_layer = Dense(
    OUTPUT_NUM_VOCAB,
    kernel_initializer = output_layer_kernel_initializer
)

In [13]:
with tf.variable_scope("decode"):
 
    training_helper = tf.contrib.seq2seq.TrainingHelper(
        inputs=decoder_input_embedded,
        sequence_length=decoder_seq_len,
        time_major=False
    )
 
    training_decoder = tf.contrib.seq2seq.BasicDecoder(
        decoder_multi_cell,
        training_helper,
        encoder_state,
        output_layer
    ) 
 
    training_decoder_output_seq, _, _ = tf.contrib.seq2seq.dynamic_decode(
        training_decoder, 
        impute_finished=True, 
        maximum_iterations=max_decoder_seq_len
    )

In [14]:
with tf.variable_scope("decode", reuse=True):
    start_tokens = tf.tile(
        tf.constant([output_voc_to_int['<GO>']], 
                    dtype=tf.int32), 
        [BATCH_SIZE], 
        name='start_tokens')
 
    # Helper for the inference process.
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
        embedding=decoder_embedding,
        start_tokens=start_tokens,
        end_token=output_voc_to_int['<EOS>']
    )
 
    # Basic decoder
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(
        decoder_multi_cell,
        inference_helper,
        encoder_state,
        output_layer
    )
 
    # Perform dynamic decoding using the decoder
    inference_decoder_output_seq, _, _ = tf.contrib.seq2seq.dynamic_decode(
        inference_decoder,
        impute_finished=True,
        maximum_iterations=max_decoder_seq_len
    )

In [15]:
# rename the tensor for our convenience
training_logits = tf.identity(training_decoder_output_seq.rnn_output, name='logits')
inference_logits = tf.identity(inference_decoder_output_seq.sample_id, name='predictions')
 
# Create the weights for sequence_loss
masks = tf.sequence_mask(
    decoder_seq_len, 
    max_decoder_seq_len, 
    dtype=tf.float32, 
    name='masks'
)
 
cost = tf.contrib.seq2seq.sequence_loss(
    training_logits,
    decoder_output_seq,
    masks
)

In [16]:
optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
 
gradients = optimizer.compute_gradients(cost)
capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var)
                        for grad, var in gradients if grad is not None]
train_op = optimizer.apply_gradients(capped_gradients)

In [17]:
def pad(xs, size, pad):
    return xs + [pad] * (size - len(xs))

In [18]:
input_seq_int = [
    [input_voc_to_int.get(voc, input_voc_to_int['<UNK>']) 
        for voc in line.split(' ')]  
    for line in input_sentences  
]
 
output_seq_int = [
    [output_voc_to_int.get(voc, output_voc_to_int['<UNK>']) 
        for voc in line.split(' ')] + [output_voc_to_int['<EOS>']]  
    for line in output_sentences  
]

# print(output_voc_to_int['<EOS>'])
print(input_seq_int[:5])
print(output_seq_int[:5])

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()  
 
for epoch in range(NUM_EPOCS + 1):  
    for batch_idx in range(len(input_seq_int) // BATCH_SIZE): 
        
        input_batch, input_lengths, output_batch, output_lengths = [], [], [], []
        for sentence in input_seq_int[batch_idx:batch_idx + BATCH_SIZE]:
           
            
            padded_voc_sent = pad(sentence, MAX_VOC_PER_LINE, input_voc_to_int['<PAD>'])
            # print(input_voc_to_int['<PAD>'])
            input_batch.append(padded_voc_sent)
            input_lengths.append(len(sentence))
            # print(padded_voc_sent)
            # break
        for sentence in output_seq_int[batch_idx:batch_idx + BATCH_SIZE]:
            padded_voc_sent = pad(sentence, MAX_VOC_PER_LINE, output_voc_to_int['<PAD>'])
            output_batch.append(padded_voc_sent)

            output_lengths.append(len(sentence))

            # print(output_voc_to_int['<PAD>'])
            # print(padded_voc_sent)
            # break
            
        # break
        _, cost_val = sess.run( 
            [train_op, cost],
            feed_dict={
                encoder_input_seq: input_batch,
                encoder_seq_len: input_lengths,
                decoder_output_seq: output_batch,
                decoder_seq_len: output_lengths,
            }
        )
        
        if batch_idx % 629 == 0:
            print('Epcoh {}. Batch {}/{}. Cost {}'.format(epoch, batch_idx, len(input_sentences) // BATCH_SIZE, cost_val))

    # break
    saver.save(sess, 'model.ckpt')   
sess.close()

[[6146, 8645, 1109, 8988, 16414, 8521, 6953, 7090, 3066, 4958, 416, 4971, 2213, 573, 5796, 14533, 3641, 12403, 15768, 6559, 12131, 16005, 2997, 13803, 2997], [14709, 11291, 6929, 14823, 8645, 12420, 15274, 6249, 7743, 11291, 16115, 5175, 368, 12318, 6249, 15408, 2997], [9361, 12131, 9597, 3066, 3576, 3066, 6598, 2600, 2997, 11540, 2997], [15408, 4971, 15078, 14784, 10919, 2997, 5175, 368, 11969, 3501, 2997, 5175, 368, 9910, 4169, 13803, 8521], [11541, 11291, 11541, 11291, 324, 368, 935, 9413, 8645, 11204, 9361, 16564, 16764, 15448, 16508]]
[[14755, 11330, 7011, 14871, 8687, 12445, 15305, 6309, 7812, 11330, 16144, 5240, 361, 12351, 6309, 15435, 3052, 3], [9382, 12171, 9609, 3115, 3625, 3115, 6668, 2645, 3052, 11584, 3052, 3], [12351, 3052, 3052, 3052, 7807, 10622, 9074, 8687, 2768, 10946, 4492, 11952, 4510, 3052, 8785, 8563, 12386, 8563, 3], [5039, 315, 3052, 3], [1597, 3052, 3]]


InvalidArgumentError: logits and labels must have the same first dimension, got logits shape [1984,16917] and labels shape [2560]
	 [[node sequence_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-15-e9174850e686>:16) ]]

Original stack trace for 'sequence_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits':
  File "d:\miniconda3\envs\py36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "d:\miniconda3\envs\py36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "d:\miniconda3\envs\py36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "d:\miniconda3\envs\py36\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "d:\miniconda3\envs\py36\lib\site-packages\ipykernel\kernelapp.py", line 612, in start
    self.io_loop.start()
  File "d:\miniconda3\envs\py36\l

In [ ]:
for batch_idx in range(len(input_sentences) // BATCH_SIZE): 
  print(batch_idx)

In [ ]:
sess = tf.InteractiveSession()    
saver.restore(sess, 'model.ckpt')

example_input_sent = "do you want to play games"
example_input_symb = [input_voc_to_int[voc] for voc in example_input_sent]
example_input_batch = [pad(example_input_symb, MAX_CHAR_PER_LINE, input_voc_to_int['<PAD>'])] * BATCH_SIZE
example_input_lengths = [len(example_input_sent)] * BATCH_SIZE

output_ints = sess.run(inference_logits, feed_dict={
    encoder_input_seq: example_input_batch,
    encoder_seq_len: example_input_lengths,
    decoder_seq_len: example_input_lengths
})[0]

output_str = ''.join([output_int_to_voc[i] for i in output_ints])
print(output_str)

In [ ]:
# Play an audio beep. Any audio URL will do.
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play